In [ ]:
from pytrials.client import ClinicalTrials
import os
import pymongo
import json
import pickle
import pprint

In [ ]:
#### Load the variables for reproducibility
with open("retrieved_studies.pkl", "rb") as f:

    retrieved_studies = pickle.load(f)

with open("mapped_data_all.pkl", "rb") as f:
    mapped_data_all = pickle.load(f)

## Connect to local MongoDB
create database and collection

In [8]:
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [20]:
db = client["ClinicalTrialsDB"]

In [21]:
final_collection = db["clinical_trial_collection"]

In [ ]:
# specify the json schema for database validation.

schema = {
    "type": "object",
    "properties": {
        "trialId": {
            "type": "string",
            "description": "A unique identifier for the clinical trial.",
        },
        "title": {
            "type": "string",
            "description": "The official title of the clinical trial.",
        },
        "startDate": {
            "type": "string",
            "pattern": "^^\\d{4}-\\d{2}-\\d{2}|NA$",
            "description": "The start date of the clinical trial.",
        },
        "endDate": {
            "type": "string",
            "pattern": "^^\\d{4}-\\d{2}-\\d{2}|NA$",
            "description": "The end date of the clinical trial, if applicable.",
        },
        "phase": {
            "type": "string",
            "enum": ["Phase 1", "Phase 2", "Phase 3", "Phase 4", "Other"],
            "description": "The phase of the clinical trial.",
        },
        "principalInvestigator": {
            "type": "array",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The name of the principal investigator.",
                },
                "affiliation": {
                    "type": "string",
                    "description": "The affiliation of the principal investigator.",
                },
            },
            "required": ["name"],
        },
        "locations": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "facility": {
                        "type": "string",
                        "description": "Name of the facility where the trial is conducted.",
                    },
                    "city": {
                        "type": "string",
                        "description": "The city where the facility is located.",
                    },
                    "country": {
                        "type": "string",
                        "description": "The country where the facility is located.",
                    },
                },
            },
        },
        "eligibilityCriteria": {
            "type": "string",
            "description": "A description of the eligibility criteria for the trial.",
        },
    },
    "required": ["trialId", "title", "phase"],
}


# validationAction: "warn"

In [ ]:
# Database validation.
db.create_collection("clinical_trial_collection", validator={"$jsonSchema": schema})

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ClinicalTrialsDB'), 'clinical_trial_collection')

In [27]:
client.list_database_names()

['ClinicalTrialsDB', 'admin', 'config', 'local']

In [23]:
db.list_collection_names()

['clinical_trial_collection']

## Retrive metadata via clinicaltrials.gov API

In [28]:
ct = ClinicalTrials()

In [ ]:
retrieved_studies = ct.get_full_studies(
    search_expr="AREA[LastUpdatePostDate]RANGE[2024-10-20, 2024-10-21]",  # 更靈活?
    max_studies=1000,  # more than 1000?
    fmt="json",
)

In [30]:
print(json.dumps(retrieved_studies["studies"][0], indent=4))

{
    "protocolSection": {
        "identificationModule": {
            "nctId": "NCT05945927",
            "orgStudyIdInfo": {
                "id": "ML44675"
            },
            "organization": {
                "fullName": "Hoffmann-La Roche",
                "class": "INDUSTRY"
            },
            "briefTitle": "A Study to Evaluate the Safety and Effectiveness of Trastuzumab Emtansine (T-DM1) as Therapy in Chinese Participants With HER2 Positive Advanced Breast Cancer",
            "officialTitle": "An Observational Study to Evaluate the Safety and Effectiveness of Trastuzumab Emtansine (T-DM1) as Second- or Later-Line Therapy in Chinese Patients With HER2 Positive Advanced Breast Cancer"
        },
        "statusModule": {
            "statusVerifiedDate": "2024-10",
            "overallStatus": "RECRUITING",
            "expandedAccessInfo": {
                "hasExpandedAccess": false
            },
            "startDateStruct": {
                "date": "2023-0

In [31]:
# Check how many studies are retrieved
len(retrieved_studies["studies"])

619

In [32]:
def map_data(input_data):
    transformed_data = []

    for study in input_data["studies"]:
        protocol = study.get("protocolSection", {})

        phase = protocol.get("designModule", {}).get("phases", "NA")
        phase_value = (
            "Other"
            if phase == "NA"
            else next(
                (
                    f"Phase {p[-1]}"
                    for p in phase
                    if p.upper() in ["PHASE1", "PHASE2", "PHASE3", "PHASE4"]
                ),
                "Other",
            )
        )

        startDate_value = str(
            protocol.get("statusModule", {})
            .get("startDateStruct", {})
            .get("date", "NA")
        )
        endDate_value = str(
            protocol.get("statusModule", {})
            .get("completionDateStruct", {})
            .get("date", "NA")
        )

        def modify_date(date):
            if len(date) == 7:
                return date + "-01"
            else:
                return date

        overall_officials = protocol.get("contactsLocationsModule", {}).get(
            "overallOfficials", []
        )
        PRINCIPAL_INVESTIGATOR = [
            {
                "name": official.get("name", "NA"),
                "affiliation": official.get("affiliation", "NA"),
            }
            for official in overall_officials
            if official.get("role") == "PRINCIPAL_INVESTIGATOR"
        ]

        transformed_study = {
            "trialId": protocol.get("identificationModule", {}).get("nctId"),
            "title": protocol.get("identificationModule", {}).get(
                "officialTitle", "NA"
            ),  # if there is no official title, provide brief title instead?
            "startDate": modify_date(startDate_value),
            "endDate": modify_date(endDate_value),
            "phase": phase_value,
            "principalInvestigator": PRINCIPAL_INVESTIGATOR,
            "locations": [
                {
                    "facility": loc.get("facility", ""),
                    "city": loc.get("city", ""),
                    "country": loc.get("country", ""),
                }
                for loc in protocol.get("contactsLocationsModule", {}).get(
                    "locations", []
                )
            ],
            "eligibilityCriteria": protocol.get("eligibilityModule", {}).get(
                "eligibilityCriteria", ""
            ),
        }

        transformed_data.append(transformed_study)

    return transformed_data

In [ ]:
# Transform the input data
mapped_data_all = map_data(retrieved_studies)

In [ ]:
# Sanity check: if all studies are mapped
len(retrieved_studies["studies"]) == len(mapped_data_all)

True

In [ ]:
#### Save the variables for reproducibility
with open("retrieved_studies.pkl", "wb") as f:

    pickle.dump(retrieved_studies, f)

with open("mapped_data_all.pkl", "wb") as f:
    pickle.dump(mapped_data_all, f)

In [38]:
final_collection.insert_many(mapped_data_all)

InsertManyResult([ObjectId('673285464dd83486d5a4d339'), ObjectId('673285464dd83486d5a4d33a'), ObjectId('673285464dd83486d5a4d33b'), ObjectId('673285464dd83486d5a4d33c'), ObjectId('673285464dd83486d5a4d33d'), ObjectId('673285464dd83486d5a4d33e'), ObjectId('673285464dd83486d5a4d33f'), ObjectId('673285464dd83486d5a4d340'), ObjectId('673285464dd83486d5a4d341'), ObjectId('673285464dd83486d5a4d342'), ObjectId('673285464dd83486d5a4d343'), ObjectId('673285464dd83486d5a4d344'), ObjectId('673285464dd83486d5a4d345'), ObjectId('673285464dd83486d5a4d346'), ObjectId('673285464dd83486d5a4d347'), ObjectId('673285464dd83486d5a4d348'), ObjectId('673285464dd83486d5a4d349'), ObjectId('673285464dd83486d5a4d34a'), ObjectId('673285464dd83486d5a4d34b'), ObjectId('673285464dd83486d5a4d34c'), ObjectId('673285464dd83486d5a4d34d'), ObjectId('673285464dd83486d5a4d34e'), ObjectId('673285464dd83486d5a4d34f'), ObjectId('673285464dd83486d5a4d350'), ObjectId('673285464dd83486d5a4d351'), ObjectId('673285464dd83486d5a4d3

In [ ]:
# Sanity check: if all studies are inserted
len(retrieved_studies["studies"]) == len(
    mapped_data_all
) == final_collection.count_documents({}) == 619

True

In [ ]:
# Sanity check
mapped_data_all[10]

{'trialId': 'NCT06649591',
 'title': 'A Survey Based Study Assessing the Feasibility of Using Standardized Clinical Vignettes to Aid in Medical Decision in Patients with Malignant Brain Tumors',
 'startDate': '2017-07-01',
 'endDate': '2025-12-01',
 'phase': 'Other',
 'principalInvestigator': [{'name': 'Marie Roguski, MD MPH',
   'affiliation': 'Tufts Medical Center'}],
 'locations': [{'facility': 'Tufts Medical Center',
   'city': 'Boston',
   'country': 'United States'}],
 'eligibilityCriteria': 'Inclusion Criteria:\r\n\r\n  -  consecutive patients treated at a single center between April 2018 and July 2023 for\r\n     malignant brain tumors, including glioma, metastasis, and lymphoma.\r\n\r\nExclusion Criteria:\r\n\r\n  -  Patients without available MRI dicom images\r\n\r\n  -  Patients with other CNS malignancies\r\n\r\n  -  Patients with multiply recurrent gliomas undergoing treatment for primarily\r\n     palliative purposes\r\n\r\n  -  Patients younger than 18 years old',
 '_id'

In [ ]:
# Sanity check
mapped_data_all[15]

{'trialId': 'NCT04431960',
 'title': 'Blackcurrant Modifies Gut Microbiota and Reduces the Risk of Postmenopausal Osteoporosis and Cardiovascular Disease: A Pilot Randomized Clinical Trial',
 'startDate': '2021-07-20',
 'endDate': '2022-10-03',
 'phase': 'Phase 1',
 'principalInvestigator': [{'name': 'Ock K Chun, PhD',
   'affiliation': 'University of Connecticut'}],
 'locations': [{'facility': 'University of Connecticut Department of Nutritional Sciences and Kinesiology Human Performance Laboratory',
   'city': 'Storrs',
   'country': 'United States'}],
 'eligibilityCriteria': 'Inclusion Criteria:\r\n\r\n  -  perimenopausal or early postmenopausal women aged 45-60 years old\r\n\r\n  -  not on HRT for at least one year before the initiation of the study\r\n\r\n  -  maintaining normal exercise level (<7 h/wk) and willing to avoid exercise 24-h prior\r\n     to blood and stool sampling and 12-h prior to bone measurements\r\n\r\n  -  willing to ingest a dietary BC supplement or placebo (u

#### Sanity checks for database

In [ ]:
# Sanity check
p1 = final_collection2.count_documents({"phase": "Phase 1"})


p2 = final_collection2.count_documents({"phase": "Phase 2"})


p3 = final_collection2.count_documents({"phase": "Phase 3"})


p4 = final_collection2.count_documents({"phase": "Phase 4"})


o = final_collection2.count_documents({"phase": "Other"})


print(p1, p2, p3, p4, o)


print(p1 + p2 + p3 + p4 + o)

90 96 61 23 349
619


In [ ]:
for trial in final_collection.find({"title": {"$regex": "Diabete", "$options": "i"}}):
    pprint.pprint(trial.get("trialId"))


final_collection.count_documents({"title": {"$regex": "Diabete", "$options": "i"}})

'NCT06649773'
'NCT04745572'
'NCT06650969'
'NCT05910840'
'NCT05743244'
'NCT05963022'
'NCT04413357'
'NCT05026424'
'NCT03835312'
'NCT02806700'
'NCT04914559'
'NCT05254002'
'NCT06650007'
'NCT06579105'


14

In [ ]:
for trial in final_collection.find({"title": {"$regex": "HER2", "$options": "i"}}):
    pprint.pprint(trial.get("trialId"))


final_collection.count_documents({"title": {"$regex": "HER2", "$options": "i"}})

'NCT05945927'
'NCT04513665'
'NCT06065748'
'NCT03901339'
'NCT03280563'
'NCT03587740'
'NCT05425550'
'NCT02007512'
'NCT06364410'
'NCT06650332'
'NCT02297438'


11

In [ ]:
final_collection.count_documents(
    {"startDate": {"$gte": "2020-01-01", "$lte": "2024-12-31"}}
)

455

In [ ]:
final_collection.count_documents(
    {
        "locations": {
            "$elemMatch": {
                "facility": {"$regex": "Orange County Research Center", "$options": "i"}
            }
        }
    }
)

3

In [ ]:
final_collection.count_documents(
    {"principalInvestigator": {"$elemMatch": {"name": "Marie Roguski, MD MPH"}}}
)

1

In [ ]:
# Get a distinct list of all principal investigator names
distinct_principal_investigators = final_collection.distinct(
    "principalInvestigator.name"
)

print("Distinct Principal Investigators:")
for name in distinct_principal_investigators:
    print(name)

Distinct Principal Investigators:
Aaron Folsom, MD, MPH
Aaron Shafer, MD
Abdusalom Abdurakhmanov, MD
Abraham Wu, MD
Adeyemi Ogunleye, MD
Afton Hassett
Ajay Premkumar, MD
Alain SAAD
Alain SAAD, MD
Alaine E Reschke-Hernandez, PhD
Alaine E. Reschke-Hernandez, PhD
Alethea Desrosiers
Alexis MOSCA, MD
Alice L Yu
Allison Gibson, PhD
Amanda Rao, PhD
Ana Oaknin, MD
Anders Thorell, Professor
Andreas M Fritzen, Associate Professor
Andreas Peyrl, MD
Andreas Trojan, Prof. Dr. med.
Andrew Kneebone
Ann Smeets, MD,PhD
Anna De Simoni, PhD
Anna GEMAHLING, MD
Anna H. Grummon, PhD
Anna Maria Hibbs, MD, MSCE
Anne-Sophie Brazeau, PhD
Annette DeVito Dabbs, PhD
Anxo Fernandez-Ferreiro, PhD
Audrey Harkness, PhD
Aung Naing, Md
Boris Bogov, Prof, PhD
Bouthaina S Dabaja
Brian Helfand, M.D.
Brian Vickery, MD
CORALIE NOEL
Carla S Stover, Ph.D.
Carlos A Zarate, M.D.
Carlos Chaccour
Carolyn Bramante, MD
Carrie L Pistenmaa, MD, MS
Cavan Reilly, PhD
Channing Paller, MD
Chao-Chi Ho, MD, PhD
Chaohui Yu, Doctor
Chaosu Hu


In [ ]:
# Aggregation pipeline to count occurrences of each principal investigator name
pipeline = [
    {"$unwind": "$principalInvestigator"},
    {"$group": {"_id": "$principalInvestigator.name", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},  # Sort by count in descending order
]

# Run the aggregation pipeline
results = final_collection.aggregate(pipeline)

# Print the results
print("Number of documents for each Principal Investigator:")
for result in results:
    name = result["_id"]
    count = result["count"]
    print(f"{name}: {count}")

Number of documents for each Principal Investigator:
Hrishikesh Chakraborty: 3
Chaosu Hu: 2
Dongmei Ji: 2
Maryam Fouladi, MD: 2
William Gleason, OD: 2
Xiaojing Hu, PhD: 2
Shounak Majumder, M.D.: 2
Brian Helfand, M.D.: 1
Lily A Brown, PhD: 1
Daniel Jones, MBBS, PhD: 1
Prof Andrew Prendergast, DPhil MRCPCH: 1
Danijela Trifunovic-Zamklar, MD, PhD: 1
Deepa G Manwani, M.D: 1
Channing Paller, MD: 1
Imran K Niazi, MD: 1
Hamiyet Kızıl, Phd RN: 1
Vikky Makker, MD: 1
Vladimir Hodzhev, Prof, PhD: 1
Shivaani Kummar, MD: 1
Thorsten Kahnt, Ph.D.: 1
Ann Smeets, MD,PhD: 1
Kenneth Offit, MD, MPH: 1
Marleen Kok, MD: 1
Frank A Bucci, Jr., MD: 1
Kevin Knight, PhD: 1
Lars O Cardell, Professor: 1
Georgi Momekov, Prof PhD: 1
Terence T. Sio, MD, MS: 1
Hirva Mamdani, MD: 1
Drew Sayer, PhD: 1
Lakshmi Koyyalagunta: 1
Craig Crandall, PhD: 1
James Yu: 1
Paola Azucena Alvarado Pelayo, Bachelor: 1
Li Huo, MD: 1
Kristie A Blum: 1
Ruth Blanco Rojo, PhD: 1
Megan Gunnar, PhD: 1
Mohamed Hamdy, Lecturer: 1
Ricardo Espinoz

In [ ]:
final_collection.count_documents(
    {
        "locations": {
            "$elemMatch": {
                "facility": {"$regex": "Orange County Research Center", "$options": "i"}
            }
        }
    }
)

3

In [ ]:
final_collection.count_documents(
    {"locations": {"$elemMatch": {"facility": {"$regex": "Orange", "$options": "i"}}}}
)

13

In [ ]:
for trial in final_collection.find(
    {"locations": {"$elemMatch": {"facility": {"$regex": "Orange", "$options": "i"}}}}
):
    for location in trial.get("locations", []):
        pprint.pprint(location["facility"])

"Children's Hospital of Alabama"
'University of Alabama at Birmingham Cancer Center'
'USA Health Strada Patient Care Center'
'Phoenix Childrens Hospital'
'Banner University Medical Center - Tucson'
"Arkansas Children's Hospital"
'University of Arkansas for Medical Sciences'
'Kaiser Permanente Downey Medical Center'
'Loma Linda University Medical Center'
"Miller Children's and Women's Hospital Long Beach"
"Children's Hospital Los Angeles"
'Cedars Sinai Medical Center'
"Valley Children's Hospital"
"UCSF Benioff Children's Hospital Oakland"
'Kaiser Permanente-Oakland'
"Children's Hospital of Orange County"
"Lucile Packard Children's Hospital Stanford University"
'Sutter Medical Center Sacramento'
'University of California Davis Comprehensive Cancer Center'
"Rady Children's Hospital - San Diego"
'Naval Medical Center -San Diego'
'UCSF Medical Center-Parnassus'
'UCSF Medical Center-Mission Bay'
'Santa Barbara Cottage Hospital'
"Children's Hospital Colorado"
"Rocky Mountain Hospital for Chil

In [ ]:
# Find documents where a principal investigator name contains "wu" (case-insensitive)
for trial in final_collection.find(
    {
        "principalInvestigator": {
            "$elemMatch": {"name": {"$regex": "wu", "$options": "i"}}
        }
    }
):
    # Loop through each principal investigator in the trial
    for investigator in trial.get("principalInvestigator", []):
        # Loop through each location in the trial and print investigator name with facility
        for location in trial.get("locations", []):
            pprint.pprint(
                (investigator["name"], location.get("facility", "No Facility"))
            )

('Ivan Wu', 'University of Minnesota')
('Abraham Wu, MD', 'Memorial Sloan Kettering Cancer Center')


In [57]:
final_collection.count_documents({"endDate": {"$lt": "2024-11-11"}})

179

In [56]:
final_collection.count_documents({"endDate": {"$gte": "2024-11-11"}})

440

In [ ]:
number = final_collection.count_documents({"endDate": "NA"})

print(number)

for trial in final_collection.find({"endDate": "NA"}):
    pprint.pprint(trial["trialId"])

4
'NCT00233272'
'NCT06649669'
'NCT00342888'
'NCT06597747'


In [55]:
final_collection.find_one({"trialId": "NCT06649669"})

{'_id': ObjectId('673285464dd83486d5a4d3f5'),
 'trialId': 'NCT06649669',
 'title': '[Trial of device that is not approved or cleared by the U.S. FDA]',
 'startDate': 'NA',
 'endDate': 'NA',
 'phase': 'Other',
 'principalInvestigator': [],
 'locations': [],
 'eligibilityCriteria': ''}

In [51]:
for trial in final_collection.find(
    {"title": "[Trial of device that is not approved or cleared by the U.S. FDA]"}
):
    # Print the trialId directly
    pprint.pprint(trial.get("trialId"))

'NCT06649669'
'NCT06597747'


In [ ]:
for trial in final_collection.find(
    {"trialId": {"$in": ["NCT06561048", "NCT06399289", "NCT06415344"]}}
):

    print("trialId:", trial.get("trialId"))

    print("eligibilityCriteria:", trial.get("eligibilityCriteria"))

    print("\n")

trialId: NCT06561048
eligibilityCriteria: Inclusion Criteria:

  1. Adult participants ≥18 years of age on the day of signing the informed consent form.

  2. Eastern Cooperative Oncology Group (ECOG) Performance Status of 0 to 2.

  3. Histologically confirmed PTCL-NOS, FHTCLs or sALCL per The International Consensus
     Classification of Mature Lymphoid Neoplasms.

  4. Progressed on, be refractory to, relapsed, or intolerant to standard therapy for
     their cancer. At least 1 but not more than 3 prior systemic therapies.

  5. Fluorodeoxyglucose-avid disease by positron emission tomography and measurable
     disease of at least 1.5 cm by computed tomography, as assessed by the site
     radiologist.

  6. Life expectancy >12 weeks.

  7. Adequate organ function as determined by:

       -  Absolute neutrophil count ≥ 1.0×10^9/L (1000/mm3) (without receiving
          granulocyte-colony stimulating factor)

       -  Platelet count ≥ 50×10^9/L (without transfusion)

       -  Hem

# Extract information from the eligibilityCriteria field in each study

In [ ]:
# import openai
from openai import OpenAI

In [169]:
from config import api_key

In [ ]:
client = OpenAI(api_key=api_key)

In [ ]:
def extract_diseases_from_inclusion_criteria(eligibility_text):
    # Construct a prompt to extract diseases/conditions only from inclusion criteria
    prompt = f"""
    Extract all diseases or conditions mentioned only in the 'Inclusion criteria' part of this eligibility criteria text:
    
    Eligibility Criteria: {eligibility_text}
    
    Please list only the diseases or conditions without any extra text.
    """

    # Call the OpenAI API with the prompt
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You're a helpful clinical trial assistant"},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0,
    )

    # Extract the response text containing the diseases/conditions
    extracted_conditions = response.choices[0].message.content
    return extracted_conditions

In [194]:
# Sample eligibility criteria text
eligibility_text = """
eligibilityCriteria: Inclusion Criteria:

  -  Male Chinese subjects aged ≤ 70 years

  -  Subjects with documented severe or moderately severe hemophilia B (FIX activity of ≤
     2%)

  -  Subjects have received FIX products for ≥ 150 exposure days (EDs) (subjects aged ≥ 6
     years) or ≥ 50 EDs (subjects aged < 6 years)

  -  Subjects have no confirmed prior history of FIX inhibitor formation

Exclusion Criteria:

  -  Known hypersensitivity (allergic reaction or anaphylaxis) to any FIX product or
     hamster protein.

  -  Known congenital or acquired coagulation disorder other than congenital FIX
     deficiency.

  -  Currently receiving intravenous (IV) immunomodulating agents such as immunoglobulin
     or chronic systemic corticosteroid treatment.

  -  Currently receiving a long-acting recombinant FIX treatment such as coagulation
     factor IX (recombinant), Fc fusion protein (Alprolix®).

  -  Use of traditional or herbal Chinese medicine(s) with an impact on hemophilia,
     including coagulation, within 28 days before Day 1 and / or refusal to abstain from
     these during the study until the end of the subject's participation in the study.

"""

# Run the extraction function
diseases_in_inclusion = extract_diseases_from_inclusion_criteria(eligibility_text)
print("Diseases/Conditions in Inclusion Criteria:", diseases_in_inclusion)

Diseases/Conditions in Inclusion Criteria: - Severe or moderately severe hemophilia B
- FIX inhibitor formation
- Congenital FIX deficiency
